In [22]:
import pandas as pd
import numpy as np

In [23]:
import os
cwd = os.getcwd()
print(cwd)

/Users/rosamund/Documents/GitHub/uk-electricity-map


In [24]:
df1 = pd.read_csv((cwd + "/uk-map.csv"))
df1.head(3)

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
0,Aberdare District Energy,EDF Energy,Gas,NaN,10.0,51.697527,-3.416181,No,NaN,2002.0,2005,2014
1,Aberthaw B,RWE Npower Plc,Coal,NaN,1455.0,51.387006,-3.406215,No,NaN,1971.0,2004,2007
2,Aberthaw B,RWE Npower Plc,Coal,NaN,1586.0,51.387006,-3.406215,No,NaN,1971.0,2008,2017


In [25]:
df1.tail(3)

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
6799,Westhampnett Solar Farm Battery Storage,West Sussex County Council (100%),Storage,Battery,4.4,50.8469,-0.745574,Yes,No,2018.0,2018,2019
6800,Willenhall Battery Storage Demonstration,University of Sheffield (100%),Storage,Battery,1.0,52.5778,-2.049130,Yes,No,2016.0,2016,2019
6801,Willersey Energy Storage Facility,Belectric (100%),Storage,Battery,1.4,52.0655,-1.849880,Yes,No,2017.0,2017,2019


In [26]:
#think appearance of excess rows may due to Simon's treament of the data carrying across
#update: it's an issue with how Simon exported the CSV at row 1770

df1.shape

(6802, 12)

In [27]:
df1.tail(3)

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
6799,Westhampnett Solar Farm Battery Storage,West Sussex County Council (100%),Storage,Battery,4.4,50.8469,-0.745574,Yes,No,2018.0,2018,2019
6800,Willenhall Battery Storage Demonstration,University of Sheffield (100%),Storage,Battery,1.0,52.5778,-2.049130,Yes,No,2016.0,2016,2019
6801,Willersey Energy Storage Facility,Belectric (100%),Storage,Battery,1.4,52.0655,-1.849880,Yes,No,2017.0,2017,2019


In [28]:
df2 = df1.rename(index=str, columns={"Site": "site", "Company": "operator", "Fuel": "type", "Type": "fuelDetail", "Capacity (MW)": "capacity", "Latitude": "latitude", "Longitude": "longitude", "Low-Carbon": "lowCarbon", "CHP": "chp", "Year Open": "yearOpen", "yearStart": "yearStart", "yearEnd": "yearEnd"})
df2.head(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
0,Aberdare District Energy,EDF Energy,Gas,NaN,10.0,51.697527,-3.416181,No,NaN,2002.0,2005,2014
1,Aberthaw B,RWE Npower Plc,Coal,NaN,1455.0,51.387006,-3.406215,No,NaN,1971.0,2004,2007
2,Aberthaw B,RWE Npower Plc,Coal,NaN,1586.0,51.387006,-3.406215,No,NaN,1971.0,2008,2017


In [29]:
df2.tail(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
6799,Westhampnett Solar Farm Battery Storage,West Sussex County Council (100%),Storage,Battery,4.4,50.8469,-0.745574,Yes,No,2018.0,2018,2019
6800,Willenhall Battery Storage Demonstration,University of Sheffield (100%),Storage,Battery,1.0,52.5778,-2.049130,Yes,No,2016.0,2016,2019
6801,Willersey Energy Storage Facility,Belectric (100%),Storage,Battery,1.4,52.0655,-1.849880,Yes,No,2017.0,2017,2019


In [30]:
#find out if any nulls for long/lat
sum(df2['longitude'].isna())

69

In [31]:
#drop rows where longitude is blank

df2 = df2.dropna(subset=['longitude'])
sum(df2['longitude'].isna())

0

In [32]:
#check latitude doesn't have NaN either

sum(df2['latitude'].isna())

0

In [33]:
#shape as expected
df2.shape

(6733, 12)

In [34]:
#replace gaps with hyphens

df2 = df2.fillna("-")
df2.head(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
0,Aberdare District Energy,EDF Energy,Gas,-,10.0,51.697527,-3.416181,No,-,2002,2005,2014
1,Aberthaw B,RWE Npower Plc,Coal,-,1455.0,51.387006,-3.406215,No,-,1971,2004,2007
2,Aberthaw B,RWE Npower Plc,Coal,-,1586.0,51.387006,-3.406215,No,-,1971,2008,2017


In [35]:
#sort by capacity so smallest drawn last

df3 = df2.sort_values(by=['capacity'], ascending=False)

df3.head(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
84,Drax,Drax Power Ltd,Coal,-,3870.0,53.737546,-0.999021,No,-,1974,2004,2012
86,Drax coal,Drax Power Ltd,Coal,-,3225.0,53.737546,-0.999021,No,-,1974,2013,2013
373,Hinkley Point C,EDF Energy,Nuclear,-,3200.0,51.206000,-3.144000,Yes,-,2026,2019,2019


In [36]:
df3.tail(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
2982,JB Wheaton,E.On,Storage,Battery,0.005,50.840997,-2.935433,Yes,-,2016,2016,2019
2965,Flat Holm Microgrid Project,Wind & Sun,Storage,Battery,0.005,51.378056,-3.122498,Yes,-,2006,2006,2019
184,Killingholme,Centrica,Gas,Combined cycle,0.000,53.653650,-0.255910,No,-,1994,2016,2016


In [37]:
#begin process of converting to geojson

import geopandas as gpd

In [38]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [39]:
geometry = [Point(xy) for xy in zip(df3.longitude, df3.latitude)]
df3 = df3.drop(['longitude', 'latitude'], axis=1)
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(df3, crs=crs, geometry=geometry)
gdf.head(3)

,site,operator,type,fuelDetail,capacity,lowCarbon,chp,yearOpen,yearStart,yearEnd,geometry
84,Drax,Drax Power Ltd,Coal,-,3870.0,No,-,1974,2004,2012,POINT (-0.9990209999999999 53.73754599999999)
86,Drax coal,Drax Power Ltd,Coal,-,3225.0,No,-,1974,2013,2013,POINT (-0.9990209999999999 53.73754599999999)
373,Hinkley Point C,EDF Energy,Nuclear,-,3200.0,Yes,-,2026,2019,2019,POINT (-3.144 51.206)


In [41]:
# NB need to delete old file before or throws error

gdf.to_file("data/power_stations.json", driver="GeoJSON")